In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Datasets/Train'
valid_path = '/content/drive/MyDrive/Datasets/Test'

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


for layer in vgg.layers:
  layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/Datasets/Train*')
  
x = Flatten()(vgg.output)
prediction = Dense(3, activation='softmax')(x)

In [ ]:
len(folders)

1

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
10/10 [==============================] - 138s 15s/step - loss: 1.1929 - accuracy: 0.5055 - val_loss: 1.1239 - val_accuracy: 0.3633
Epoch 2/10
10/10 [==============================] - 8s 789ms/step - loss: 0.6476 - accuracy: 0.7920 - val_loss: 0.8894 - val_accuracy: 0.6367
Epoch 3/10
10/10 [==============================] - 8s 787ms/step - loss: 0.2501 - accuracy: 0.9623 - val_loss: 0.6260 - val_accuracy: 0.7733
Epoch 4/10
10/10 [==============================] - 8s 792ms/step - loss: 0.1464 - accuracy: 0.9946 - val_loss: 0.7213 - val_accuracy: 0.7333
Epoch 5/10
10/10 [==============================] - 8s 793ms/step - loss: 0.1199 - accuracy: 0.9914 - val_loss: 0.4282 - val_accuracy: 0.8667
Epoch 6/10
10/10 [==============================] - 8s 799ms/step - loss: 0.0782 - accuracy: 1.0000 - val_loss: 0.8936 - val_accuracy: 0.6400
Epoch 7/10
10/10 [==============================] - 8s 797ms/step - loss: 0.0607 - accuracy: 0.9991 - val_loss: 0.6776 - val_accuracy: 0.7833
Epoch 

In [ ]:
model.save('facefeatures_new_model.h5')
